# Simulate measurements from V1, V2 and V3 waveforms

Starting with a `.npy`-representation of source (patch) activity made with [this notebook](Make%20V1-V3%20waveforms.ipynb), forward model data for different stimulus locations.

In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from scipy.misc import imread

## Load the source data

In [3]:
imname = 'Hagler2014-Fig8D-cropped.npy'
fs = 250.  # Hz

In [6]:
# n_times x 4 (= [t_axis, V1, V2, V3])
waveforms = np.load(imname.split('.')[0] + '.npy')

## Load the forward data

In [7]:
from retinotopic_helpers import *
from mne.io import read_raw_fif

In [11]:
# ch_type = 'meg'  # may make little sense, since scale of 'grad' dominates!
# ch_type = 'grad'
ch_type = 'mag'

regions = ['V1', 'V2', 'V3']

In [8]:
data_path = '/Users/cjb/projects/CFF_Retinotopy/scratch'
subject = '030_WAH'
subjects_dir = join(data_path, 'fs_subjects_dir')
fname_fwd = join(data_path, 'restricted_1LBEM-fwd.fif')
fname_raw = join(data_path, 'VS1_cropped1sec.fif')
info = read_raw_fif(fname_raw, preload=False).info

Opening raw data file /Users/cjb/data/MINDLAB2013_01-MEG-AttentionEmotionVisualTracking/scratch/VS1_cropped1sec.fif...
This filename (/Users/cjb/data/MINDLAB2013_01-MEG-AttentionEmotionVisualTracking/scratch/VS1_cropped1sec.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
    Range : 166000 ... 167000 =    166.000 ...   167.000 secs
Ready.
Current compensation grade : 0


<ipython-input-8-535f17674584>:6: RuntimeWarning: This filename (/Users/cjb/data/MINDLAB2013_01-MEG-AttentionEmotionVisualTracking/scratch/VS1_cropped1sec.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  info = read_raw_fif(fname_raw, preload=False).info


In [12]:
fwd = read_forward_solution(fname_fwd)
# change to surface coords & select channel type(s)
fwd = prepare_gain(fwd, ch_type=ch_type)
fwd_fixed = convert_forward_solution(fwd, force_fixed=True, copy=True)

Reading forward solution from /Users/cjb/projects/CFF_Retinotopy/scratch/restricted_1LBEM-fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (9700 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    Cartesian source orientations...
[done]
    102 out of 306 channels remain after picking
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


In [ ]:
evoked = simulate_evoked(fwd_fixed, stc, info, cov=cov, snr=snr)